# Polling data capture from Wikipedia

**Purpose:**
 * Capture data from the Wiki Page on Opinion Polling
 * Adjust that data for anomalies (for example, ensureing voting intention sums to 100%)
 * Save the data as a bsis for further analysis

**Make sure to:**
 * run before doing any analysis; and
 * check the data validation before moving on to the analysis.

## Python setup

In [1]:
# analytic imports
import pandas as pd
from IPython.display import display

In [2]:
# local imports
import data_capture as dc
from common import ATTITUDINAL, VOTING_INTENTION, NSW, VIC, QLD, SA, WA, TAS, NT

In [3]:
STATES = [NSW, VIC, QLD, SA, WA, TAS, NT]

## Get raw polling data from Wikipedia

### Get all tables from the Wikipedia web page

In [4]:
def get_tables(verbose: bool = False) -> dict[str, pd.DataFrame]:
    """
    Get Opinion Polling Tables for the 2025 Australian Federal Election
    from the Wikipedia page.

    Verbose option will print the numbered tables - sometimes needed when
    Wiki page has been reorganised.

    Note: web-scraping is fragile, and depends on the page maintaining the
    same structure.  This function will need to be updated if the Wikipedia
    page is reorganised.
    """

    # capture the table-list from the Wikipedia page
    url = (
        "https://en.wikipedia.org/wiki/"
        + "Opinion_polling_for_the_2025_Australian_federal_election"
    )
    df_list = dc.get_table_list(url)
    print(f"Total number of tables on page: {len(df_list)}")

    if verbose:
        # show each table's columns
        for i, frame in enumerate(df_list):
            print(f"{i}: {frame.columns}\n")

    # The Wikipedia table numbers will need updating each year ...
    # And whenever the Wikipedia page is reorganised (such that the tables are renumbered)
    voting_tables = (3, 4, 5, 6)
    attitudinal_tables = (7, 8, 9, 10)
    nsw = (14,)
    vic = (17,)
    qld = (20,)
    wa = (23,)
    sa = (26,)
    tas = (28,)
    nt = (30,)

    prep = {
        VOTING_INTENTION: voting_tables, 
        ATTITUDINAL: attitudinal_tables,
        NSW: nsw,
        VIC: vic,
        QLD: qld,
        WA: wa,
        SA: sa,
        TAS: tas,
        NT: nt,
    }
    box = {}
    for label_, table_list in prep.items():
        print("Collating:", label_, table_list)
        table_ = dc.get_combined_table(df_list, table_list, verbose=False)
        if table_ is None:
            print(f"Failed to get {label_} table")
            continue
        table_ = table_.copy()
        table_ = dc.clean(table_)
        box[label_] = table_
        print(f"{label_}: {len(table_)} rows {table_.index}")

    return box


data = get_tables(verbose=False)

Total number of tables on page: 42
Collating: voting-intention (3, 4, 5, 6)
voting-intention: 227 rows RangeIndex(start=0, stop=227, step=1)
Collating: attitudinal (7, 8, 9, 10)
attitudinal: 129 rows RangeIndex(start=0, stop=129, step=1)
Collating: NSW (14,)
NSW: 37 rows RangeIndex(start=0, stop=37, step=1)
Collating: VIC (17,)
VIC: 37 rows RangeIndex(start=0, stop=37, step=1)
Collating: QLD (20,)
QLD: 38 rows RangeIndex(start=0, stop=38, step=1)
Collating: WA (23,)
WA: 12 rows RangeIndex(start=0, stop=12, step=1)
Collating: SA (26,)
SA: 9 rows RangeIndex(start=0, stop=9, step=1)
Collating: TAS (28,)
TAS: 2 rows RangeIndex(start=0, stop=2, step=1)
Collating: NT (30,)
NT: 3 rows RangeIndex(start=0, stop=3, step=1)


/Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/data_capture.py:155: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace("", np.nan)  # NaN empty lines
/Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/data_capture.py:155: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace("", np.nan)  # NaN empty lines
/Users/bryanpalmer/Australian-Federal-Election-2025/notebooks/data_capture.py:155: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain t

### Quick look at most recent N polls

In [5]:
def quick_look(n=2):
    """
    Display the last n rows of each table.
    """

    for label_, table_ in data.items():
        print(f"{label_}:")
        display(table_.tail(n))
        print()


quick_look()

voting-intention:


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,Primary vote UND,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
225,17–19 January 2025,Freshwater Strategy,Online,NaN,40.0,32.0,13.0,NaN,NaN,15.0,NaN,49.0,51.0,2025-01-17,2025-01-18,2025-01-19
226,15–21 January 2025,Resolve Strategic,Online,NaN,38.0,27.0,13.0,7.0,NaN,16.0,NaN,48.0,52.0,2025-01-15,2025-01-18,2025-01-21



attitudinal:


,Date,Firm,Interview mode,Sample,Preferred prime minister Albanese,Preferred prime minister Dutton,Preferred prime minister Don't Know,Preferred prime minister Net,Albanese Satisfied,Albanese Dissatisfied,Albanese Don't Know,Albanese Net,Dutton Satisfied,Dutton Dissatisfied,Dutton Don't Know,Dutton Net,First Date,Mean Date,Last Date
127,17–19 January 2025,Freshwater Strategy,Online,NaN,43.0,43.0,14.0,0.0,32.0,50.0,18.0,-18.0,36.0,40.0,24.0,-4.0,2025-01-17,2025-01-18,2025-01-19
128,15–21 January 2025,Resolve Strategic,Online,NaN,34.0,39.0,27.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-15,2025-01-18,2025-01-21



NSW:


...

,Date,Firm,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,Primary vote UND,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
7,15 July – 20 September 2024,Newspoll,374.0,35.0,36.0,9.0,10.0,NaN,10.0,NaN,54.0,46.0,2024-07-15,2024-08-17,2024-09-20
8,7 October – 6 December 2024,Newspoll,280.0,37.0,35.0,9.0,7.0,NaN,12.0,NaN,53.0,47.0,2024-10-07,2024-11-06,2024-12-06



TAS:


,Date,Firm,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote JLN,Primary vote UAP,Primary vote IND,Primary vote OTH,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
0,21 June 2022,Roy Morgan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,37.0,2022-06-21,2022-06-21,2022-06-21
1,28 August – 12 October 2023,Newspoll,366.0,25.0,30.0,13.0,NaN,NaN,NaN,27.0,57.0,43.0,2023-08-28,2023-09-19,2023-10-12



NT:


,Date,Brand,Sample Size,Seat Tally L/NP,Seat Tally ALP,Seat Tally GRN,Seat Tally OTH,Most Likely Outcome,First Date,Mean Date,Last Date
1,10 July - 27 August 2024,Accent Research/RedBridge Group,5976.0,68,69,3,10,Labor Minority,2024-07-10,2024-08-03,2024-08-27
2,29 October - 20 November 2024,Accent Research/RedBridge Group,4909.0,71,65,4,10,Coalition Minority,2024-10-29,2024-11-09,2024-11-20


### Standardise column names

In [6]:
def fix_col_names(input_: dict[str, pd.DataFrame]) -> dict[str, pd.DataFrame]:
    """
    Standardise the column names in the tables. For some reason,
    the voting intention and attitudinal tables at Wikipedia have
    different column names for the same information.
    """

    fix = {
        # from : to
        "Firm": "Brand",
        "Sample": "Sample size",
    }

    output = {}
    for label_, table_ in input_.items():
        for old_col, new_col in fix.items():
            fix_me_list = table_.columns[
                table_.columns.str.contains(old_col, case=False)
            ]
            if len(fix_me_list) == 1:
                fix_me_string = fix_me_list[0]
                table_ = table_.rename(columns={fix_me_string: new_col})
                print(f"{label_} fixed col from {fix_me_string} to {new_col}")
                output[label_] = table_

    return output


data = fix_col_names(data)
data.keys()

voting-intention fixed col from Sample size to Sample size
attitudinal fixed col from Firm to Brand
attitudinal fixed col from Sample to Sample size
NSW fixed col from Firm to Brand
NSW fixed col from Sample size to Sample size
VIC fixed col from Firm to Brand
VIC fixed col from Sample size to Sample size
QLD fixed col from Firm to Brand
QLD fixed col from Sample size to Sample size
WA fixed col from Firm to Brand
WA fixed col from Sample size to Sample size
SA fixed col from Firm to Brand
SA fixed col from Sample size to Sample size
TAS fixed col from Firm to Brand
TAS fixed col from Sample size to Sample size
NT fixed col from Sample Size to Sample size


dict_keys(['voting-intention', 'attitudinal', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'NT'])

### Remove MRP polls
MRP = multi-regression post-stratification polls

In [7]:
def remove_mrp_polls(input_: dict[str, pd.DataFrame]) -> dict[str, pd.DataFrame]:
    """
    Remove aggregated 'polls' conducted by Multilevel
    Regression and Post-stratification (MRP).
    """

    output = {}


    for label_, table_ in input_.items():
        if label_ in STATES:
            # Happy to keep the MRP polls for the states
            output[label_] = table_
            continue

        # drop by Brand Name
        drop_bool = table_["Brand"].str.contains("Accent Research", na=False) & table_[
            "Brand"
        ].str.contains("RedBridge", na=False)
        drop_index = drop_bool[drop_bool].index

        if len(drop_index) > 0:
            # adjust the table
            print(f"{label_} MRP about to drop:")
            display(table_.loc[drop_index])
            table_ = table_.drop(drop_index)

        # In future we might want to drop by other criteria

        # save the table
        output[label_] = table_

    return output


data = remove_mrp_polls(data)
data.keys()

voting-intention MRP about to drop:


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,Primary vote UND,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
165,10 Jul – 27 Aug 2024,Accent Research/ RedBridge Group,Online,5976.0,38.0,32.0,12.0,NaN,NaN,18.0,NaN,50.0,50.0,2024-07-10,2024-08-03,2024-08-27
205,29 Oct – 20 Nov 2024,Accent Research/ RedBridge Group,Online,4909.0,39.0,31.0,11.0,NaN,NaN,19.0,NaN,49.0,51.0,2024-10-29,2024-11-09,2024-11-20


dict_keys(['voting-intention', 'attitudinal', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'NT'])

In [8]:
def check_for_large_samples(
    data_dict: dict[str, pd.DataFrame],
    threshold: int = 3000,
) -> None:
    """
    Check for unusually large sample sizes - may be MRP polling.
    """

    for name, t in data_dict.items():
        sample_col = t.columns[t.columns.str.contains("sample", case=False)][0]
        odd = t.index[t[sample_col].notna() & (t[sample_col] >= threshold)]
        print(odd)
        if len(odd):
            print(
                f"{name}: --CHECK-- Based on sample size, these rows might be MRP data:"
            )
            display(t.loc[odd])
            print("=" * 40)


check_for_large_samples(data)

Index([62, 172, 197], dtype='int64')
voting-intention: --CHECK-- Based on sample size, these rows might be MRP data:


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,Primary vote UND,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
62,22 Sep – 4 Oct 2023,Resolve Strategic,Online,4728.0,31.0,37.0,12.0,7.0,2.0,11.0,NaN,57.0,43.0,2023-09-22,2023-09-28,2023-10-04
172,6–29 August 2024,Wolf & Smith,Online,10239.0,36.0,29.0,13.0,6.0,NaN,15.0,NaN,51.0,49.0,2024-08-06,2024-08-17,2024-08-29
197,14–25 October 2024,ANU,Online,3622.0,38.2,31.8,11.8,NaN,NaN,NaN,9.5,50.0,50.0,2024-10-14,2024-10-19,2024-10-25


Index([0, 38], dtype='int64')
attitudinal: --CHECK-- Based on sample size, these rows might be MRP data:


,Date,Brand,Interview mode,Sample size,Preferred prime minister Albanese,Preferred prime minister Dutton,Preferred prime minister Don't Know,Preferred prime minister Net,Albanese Satisfied,Albanese Dissatisfied,Albanese Don't Know,Albanese Net,Dutton Satisfied,Dutton Dissatisfied,Dutton Don't Know,Dutton Net,First Date,Mean Date,Last Date
0,23–31 May 2022,Morning Consult,Online,3770.0,NaN,NaN,NaN,NaN,51.0,24.0,25.0,27.0,NaN,NaN,NaN,NaN,2022-05-23,2022-05-27,2022-05-31
38,29 May – 12 June 2023,CT Group,Online,3000.0,NaN,NaN,NaN,NaN,42.0,36.0,22.0,6.0,NaN,NaN,NaN,NaN,2023-05-29,2023-06-05,2023-06-12


Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([0, 1, 2], dtype='int64')
NT: --CHECK-- Based on sample size, these rows might be MRP data:


,Date,Brand,Sample size,Seat Tally L/NP,Seat Tally ALP,Seat Tally GRN,Seat Tally OTH,Most Likely Outcome,First Date,Mean Date,Last Date
0,February - May 2024,Accent Research/RedBridge Group,4040.0,60,77,3,11,Labor Majority,2024-02-01,2024-02-01,2024-02-01
1,10 July - 27 August 2024,Accent Research/RedBridge Group,5976.0,68,69,3,10,Labor Minority,2024-07-10,2024-08-03,2024-08-27
2,29 October - 20 November 2024,Accent Research/RedBridge Group,4909.0,71,65,4,10,Coalition Minority,2024-10-29,2024-11-09,2024-11-20


## Preliminary data validation

### Distribute undecideds if the pollster has not

Mostly affects the Essential poll.

In [9]:
def distribute_undecided(data_dict: dict[str, pd.DataFrame]) -> dict[str, pd.DataFrame]:
    """
    Distribute undecided voters to the primary and 2pp vote.

    Note: Essential often does not distribute undecideds to
    the 2pp Vote share.
    """

    if dc.UNDECIDED_COLUMN in data_dict[VOTING_INTENTION]:
        revised = dc.distribute_undecideds(
            table=data_dict[VOTING_INTENTION].copy(),
            col_pattern_list=["Primary vote", "2pp vote"],
        )
        revised = revised.drop(columns=dc.UNDECIDED_COLUMN)
        data_dict[VOTING_INTENTION] = revised
    else:
        print("CHECK: this step was not applied")
        print("Most likely because it has already been applied.")

    return data_dict


data = distribute_undecided(data)
data.keys()

For Primary vote distributed undecideds over 23.56% of rows.
For 2pp vote distributed undecideds over 23.56% of rows.


dict_keys(['voting-intention', 'attitudinal', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'NT'])

### Add in Primary Other if the pollster has not

In [10]:
def fix_primary_other(data_dict: dict[str, pd.DataFrame]) -> dict[str, pd.DataFrame]:
    """
    Ensure that the 'Primary vote OTH' column is data populated.
    """

    other = "Primary vote OTH"
    majors = ["L/NP", "ALP", "GRN"]
    minor_p = [
        x
        for x in data[VOTING_INTENTION].columns
        if "Primary" in x and not any(z in x for z in majors)
    ]
    major_p = [
        x
        for x in data[VOTING_INTENTION].columns
        if "Primary" in x and any(z in x for z in majors)
    ]

    rows = data[VOTING_INTENTION][minor_p].isna().sum(axis=1) == len(minor_p)
    if rows.sum() > 0:
        print("Changed from ...")
        display(data[VOTING_INTENTION].loc[rows])
        data[VOTING_INTENTION].loc[rows, other] = 100 - data[VOTING_INTENTION].loc[
            rows, major_p
        ].sum(axis=1)

        print("Changed to ...")
        display(data[VOTING_INTENTION].loc[rows])

    return data_dict


data = fix_primary_other(data)
print(data.keys())

Changed from ...


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
197,14–25 October 2024,ANU,Online,3622.0,42.63643,35.493154,13.170416,NaN,NaN,NaN,54.75,54.75,2024-10-14,2024-10-19,2024-10-25


Changed to ...


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
197,14–25 October 2024,ANU,Online,3622.0,42.63643,35.493154,13.170416,NaN,NaN,8.7,54.75,54.75,2024-10-14,2024-10-19,2024-10-25


dict_keys(['voting-intention', 'attitudinal', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'NT'])


## Forced data normalisation

Force columns that should sum to 100 to sum to 100. 
(But we only normalise if the sum is <99 or >101.)

This is a very aggressive treatment, and the rows being forced into
submission need to be considered and reflected upon from time to time.

In [11]:
# All the columns that match the CHECKABLE_100 patterns
# are checked to ensure that they sum to 100. If not, they
# are upweighted/downweighted to sum to 100.

CHECKABLE_100: dict[str, list[str]] = {
    # label: [list of regex-patterns],
    VOTING_INTENTION: [
        r"Primary",
        r"2pp",
    ],
    ATTITUDINAL: [
        r"^Dutton (Satisfied|Dissatisfied|Don't Know)",
        r"^Albanese (Satisfied|Dissatisfied|Don't Know)",
        r"Preferred [Pp]rime [Mm]inister (Dutton|Albanese|Don't Know)",
    ],
}

data = dc.normalise(data, CHECKABLE_100, verbose=False) | {k: v for k, v in data.items() if k in STATES}

print(data.keys())

19.11% of rows need normalisation - for voting-intention, Primary.
8.00% of rows need normalisation - for voting-intention, 2pp.
10.85% of rows need normalisation - for attitudinal, ^Dutton (Satisfied|Dissatisfied|Don't Know).
15.50% of rows need normalisation - for attitudinal, ^Albanese (Satisfied|Dissatisfied|Don't Know).
1.55% of rows need normalisation - for attitudinal, Preferred [Pp]rime [Mm]inister (Dutton|Albanese|Don't Know).
dict_keys(['voting-intention', 'attitudinal', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'NT'])


### Recalculate Net attitudinal

In [12]:
# still to write this function

## Manage methodology changes

In [13]:
def methodology_change(data_dict: dict[str, pd.DataFrame]) -> dict[str, pd.DataFrame]:
    """
    If a pollster firm substantially change the way in which they
    collect data we need to reflect this in the branding for the poll.
    """

    # Essential added education into its weighting
    # from the last poll in October 2023.
    effective_date = "2023-10-24"
    change_from = "Essential"
    change_to = "Essential 2"
    data_dict = dc.methodology(data_dict, effective_date, change_from, change_to)

    # Resolve Strategic appears to have changed in 2024
    effective_date = "2024-01-01"
    change_from = "Resolve Strategic"
    change_to = "Resolve Strategic 2"
    data_dict = dc.methodology(data_dict, effective_date, change_from, change_to)

    return data_dict


data = methodology_change(data)
print(data.keys())

dict_keys(['voting-intention', 'attitudinal', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'NT'])


## Final data validation

Please check any rows identified as a result of this step.

In [14]:
def final_check(data_dict: dict[str, pd.DataFrame]) -> None:
    """
    Check the final data for any anomalies.
    """

    for label, check_list in CHECKABLE_100.items():
        row_check: pd.DataFrame | None = dc.row_sum_check(
            data_dict[label], check_list, tolerance=1.01
        )
        if row_check is None or row_check.empty:
            print(f"{label} {check_list} looks good.\n")
            continue
        print(label, check_list)
        display(row_check)
        print("\n")


final_check(data)

voting-intention ['Primary', '2pp'] looks good.

attitudinal ["^Dutton (Satisfied|Dissatisfied|Don't Know)", "^Albanese (Satisfied|Dissatisfied|Don't Know)", "Preferred [Pp]rime [Mm]inister (Dutton|Albanese|Don't Know)"] looks good.



In [15]:
print(data.keys())

dict_keys(['voting-intention', 'attitudinal', 'NSW', 'VIC', 'QLD', 'WA', 'SA', 'TAS', 'NT'])


## Save the checked data

In [16]:
dc.store(data)

## All done

In [17]:
%load_ext watermark
%watermark --python --machine --conda --iversions --watermark

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

conda environment: pymc

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pandas : 2.2.3
IPython: 8.31.0

Watermark: 2.5.0



In [18]:
print("Finished")

Finished
